In [1]:
from results_imports import *
from pandas import DataFrame
import glob
import joblib

# sys.path.insert(0, os.path.abspath('../scripts'))


In [2]:
acp_base = TemporalMultiGrid(feature_file)

sp_base = TemporalMultiGrid(
    '/Users/rwspicer/Desktop/data/V1/master-project/training/SP/v1/baseline/multigrid/SP-rf-training-set.yml'
)
# acp_base_features.config['grid_name_map']

In [18]:



def get_grid_order(tmg):
    gnm = tmg.config['grid_name_map']
    gnl = ['' for i in gnm]
    for g in gnm:
        gnl[gnm[g]] = g
    return gnl

# base_features.config['grid_name_map']


hd_path = "/Volumes/toshi-stati/data/V1/"

acp_feature_dir = "master-project/training/ACP/v2/"
sp_feature_dir = "master-project/training/SP/v1/"


sa_options = [
    'no-geolocation',
    'with-labels',
    'with-random',
    'without-elevation',
    'no-lt-10',
    'no-max',
    'no-max-no-geo',
    'without-partial-precip'
]

sa_dirs = {s:os.path.join(s, 'multigrid', s + '.yml') for s in sa_options}
sa_dirs

sp_model_dir = "/Users/rwspicer/classes/masters-project/sensitivity-analysis/SP/"
acp_model_dir = "/Users/rwspicer/classes/masters-project/sensitivity-analysis/ACP/"

sp_models = glob.glob(os.path.join(sp_model_dir, '*.joblib'))
acp_models = glob.glob(os.path.join(acp_model_dir, '*.joblib'))

In [4]:
## baseline sp model is SP-rf-training-set_rfm_e50_md100_mfAUTO_mln50000_msl8_mss5_tdp75.joblib from
## SA run it was trained there inadvertently

# l_grid = TemporalGrid(
#     '/Users/rwspicer/Desktop/data/V1/thermokarst/initiation-regions/SP/v1/PDM-5var/without_predisp/multigrid/SP-TKI-PDM5.yml'
# )
# f_grid = TemporalMultiGrid(
#     '/Users/rwspicer/Desktop/data/V1/master-project/training/SP/v1/baseline/multigrid/SP-rf-training-set.yml'
# )
# hyperparameters = forest.RFParams('rfm_e50_md100_mfAUTO_mln50000_msl8_mss5_tdp.75.yml')

# features, labels, index = forest.setup(f_grid,l_grid, .75)

# sp_base_model = forest.create_model(
#                 features, 
#                 labels, 
#                 hyperparameters, 
#                 2, #verbosity
#                 12  #n_jobs
#             )
# joblib.dump(sp_base_model,'rfm_e50_md100_mfAUTO_mln50000_msl8_mss5_tdp.75.joblib')?

In [5]:
# model = joblib.load(sp_models[0])
# show_feature_importances(model, get_grid_order(sp_base))
def load_fi_results(models, data_loc):
    """
    """
    fi_s = {}
    for model_file in models:
        model = joblib.load(model_file)
        
        name = os.path.split(model_file)[1].replace('.joblib','')
        data_set = os.path.split(model_file)[1].split('_')[0]
        name = name.replace('.75', '75')
#         name = name.replace('.25', '75')
#         name = name.replace('', '75')
#         print (name)
        fi = {}
        base_name = '_'.join(name.split('_')[1:])
        if base_name == 'rfm_e50_md100_mfAUTO_mln50000_msl8_mss5_tdp75' or \
            base_name == 'rfm_e50_md100_mfAUTO_mln50000_msl8_mss5_tdp.75':
#             print('HERE')
            fi['name'] = 'baseline'
            
        else:
            base = set('rfm_e50_md100_mfAUTO_mln50000_msl8_mss5_tdp75'.split('_')) 
            base.add('tdp.75')
            change = set(name.split('_')) 
            diff = (change ^ base) - base - {'SP-rf-training-set'}
            diff = diff.pop()
            
            fi['name'] = 'baseline-' + diff
        
        
        
        
        
        if data_set in sa_options:
            data_path = os.path.join(hd_path, data_loc  ,sa_dirs[data_set])
            fi['training data'] = data_set
            fi.update(show_feature_importances(model, get_grid_order(TemporalMultiGrid(data_path))))
        else:
            fi['training data'] = 'base'
            fi.update(show_feature_importances(model, get_grid_order(sp_base)))

#         if name == 'with-random_rfm_e50_md100_mfAUTO_mln50000_msl8_mss5_tdp':
#             print (fi)

        
        
            
            
        fi_s[name] = fi
    return fi_s

In [13]:
sp_fi_dict = load_fi_results(sp_models,sp_feature_dir)

In [14]:
sp_fi_df = DataFrame(sp_fi_dict).T
sp_fi_df.to_csv('../results/sp_sa_fi_results.csv')

In [15]:
# sp_fi_df[sp_fi_df['name'] =='baseline']
sp_fi_df[sp_fi_df['training data'] == 'base'].to_csv('../results/sp_sa_fi_results_hyperparam_only.csv')

In [20]:
acp_fi_dict = load_fi_results(acp_models,acp_feature_dir)

In [21]:
acp_fi_df = DataFrame(acp_fi_dict).T
acp_fi_df.to_csv('../results/acp_sa_fi_results.csv')


In [22]:
acp_fi_df[acp_fi_df['training data'] == 'base'].to_csv('../results/acp_sa_fi_results_hyperparam_only.csv')

In [23]:
acp_fi_df[acp_fi_df['training data'] != 'base'].to_csv('../results/acp_sa_fi_results-feat.csv')